# TP3 : Algorithme de recherche de Grover

In [ ]:
# only if using Google Colab:
!pip install myqlm

In [ ]:
import numpy as np

from qat.lang import QRoutine, H, CNOT, RY, Z, X, CCNOT, Program
from qat.qpus import get_default_qpu

qpu = get_default_qpu()

def display_result(circuit, nbshots=0, idx=None):
    result = qpu.submit(circuit.to_job(nbshots=nbshots, qubits=idx))
    if nbshots:
        tmp = {}
        for sample in result:
            state = sample.state
            if not state in tmp:
                tmp[state] = 0.
            tmp[sample.state] += sample.probability
        for state, proba in tmp.items():
            print("Etat %s: probabilité %s" % (state, proba))
    else:
        for sample in result:
            print("Etat %s: probabilité %s, amplitude %s" % (sample.state, sample.probability, sample.amplitude))

# Description de l'algorithme

[L'algorithme de Grover](https://fr.wikipedia.org/wiki/Algorithme_de_Grover), proposé en 1996 par Lov Grover, est l'algorithme de recherche quantique. Il consiste à rechercher un ou plusieurs éléments suivant un critère dans une liste d'éléments **non classés** de longueur $N=2^n$. Cet algorithme a pour intérêt de résoudre le problème de recherche en un temps en $O(\sqrt N)$ au lieu de $O(N)$ que proposerait une solution classique.

L'algorithme de Grover fonctionne de la manière suivante :
1) **Superposition** : On crée une superposition sur toutes les entrées possibles dans le registre quantique d'entrée $x$ à l'aide de portes $H$,
2) **Marquage** : Un **oracle** vient marquer les éléments qui vérifient le critère donné pour la recherche,
3) **Amplification** : On amplifie l'amplitude (donc la probabilité) des états marqués à l'aide de l'**opérateur de diffusion**.

Après plusieurs répétitions des étapes de marquage et d'amplification (en pratique le nombre d'itérations optimal est d'environ $\frac{\pi}{4}\sqrt{2^n}$), on peut extraire le résultat à l'aide d'une mesure. Ci-dessous est donné le circuit quantique annoté de l'algorithme de Grover

<center>
    <img src="https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/grover_circuit.png" width=700/>
</center>

On peut visualiser l'évolution de l'algorithme en utilisant un histogramme d'amplitude comme ci-dessous. A partir de la superposition uniforme, avec l'état recherché (inconnu), on applique l'oracle qui va venir marquer cet état. Le marquage consiste à appliquer une phase de $-1$ sur l'état, son amplitude devient donc négative. Cela a pour effet de faire baisser la moyenne des amplitudes sur tous les états. Ensuite on applique l'opérateur de diffusion qui consiste à effectuer une symétrie par rapport à la moyenne des amplitudes des états. On a alors deux cas possibles pour la symétrie:

- L'état est un état non recherché : le symétrique par rapport à la moyenne fait baisser l'amplitude de cet état.
- L'état est un état recherché : le symétrique par rapport à la moyenne refait passer l'amplitude de cet état dans le positif en amplifiant sa valeur.

<center>
    <img src="https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/histogram_grover.png" width=600/>
</center>

En une itération, on a commencé à amplifier l'amplitude de l'état recherché et en contre-partie fait diminuer celles des états non-recherchés. En itérant plusieurs fois avec les étapes de marquage et d'amplification on fini par grandement amplifier l'amplitude de l'état recherché et donc la probabilité de la mesurer en sortie.

# Implémentation de l'algorithme

Nous allons dans un premier temps implémenter chaque sous-bloc de l'algorithme de Grover pour ensuite les combiner.

## Superposition

**Question 1** : Implémenter la routine quantique qui permet de créer une superposition uniforme sur $n$ qubits


In [ ]:
def superposition(n):
    rout = QRoutine()

    return rout

On vérifie que cela fonctionne bien sur $n=3$ qubits

In [ ]:
circuit = superposition(3).to_circ()
display_result(circuit)

## Marquage

Pour la suite, nous allons implémenter deux oracles de marquage différents pour tester l'algorithme. Pour appeler des portes NOT multi-contrôlées il est possible d'utiliser l'instruction suivante : 

<center>
    X.ctrl(nombre_de_qubits_de_controle)(qubits_de_ctrl, qubit_cible)
</center>

Par exemple, *X.ctrl(3)(q0,q1,q2,q3)* applique une porte $X$ sur le qubit q3, contrôlée par les qubits q0, q1 et q2.


L'objectif est à partir d'une fonction de marquage $f$ telle que 

$$
f(x) = \begin{cases} 0 & \textrm{si x n'est pas recherché} \\ 1 & \textrm{si x est recherché} \end{cases}
$$

de créer un oracle $U$ tel que

$$
U(x)= (-1)^{f(x)}x.
$$

Pour ce faire, on va travailler en deux temps:
- Créer une routine quantique $U_f$ qui implémente la transformation $\left| x \right \rangle \left| y \right \rangle \rightarrow \left| x \right \rangle \left| y \oplus f(x) \right \rangle$
- Utiliser $U_f$ pour construire $U$ qui implémente la transformation $\left| x \right \rangle \rightarrow (-1)^{f(x)}\left| x \right \rangle$


### Fonction 1

Le circuit suivant permet d'appliquer le fonction 1, avec une entrée sur 4 qubits ($x$) et une sortie sur 1 qubit ($y$).

<center>
    <img src="https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/oracle_1.png" width="300"/>
</center>

**Question 2** : A la main, retrouver les différentes valeurs de $x$ recherchées (pour lesquelles $y = 1$ en sortie)

*REPONSE*

**Question 3** : Implémenter le circuit correspondant à la fonction 1 et vérifier que le circuit généré correspond bien à celui donné ci-dessus

In [ ]:
def fonction_1():
    rout = QRoutine()

    return rout

**Question 4** : Verifier la réponse à la question 2 en simulant le circuit pour tous les valeurs possibles de $x$

### Fonction 2

Le ciruit suivant permet d'appliquer la fonction 2.

<center>
    <img src="https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/oracle_2.png" width="450"/>
</center>

**Question 5** : A la main, retrouver les différentes valeurs de $x$ recherchées (pour lesquelles $y=1$)

*REPONSE*

**Question 6** : Implémenter le circuit correspondant à la fonction 2 et vérifier que le cicruit généré correpsond bien à celui ci-dessus

In [ ]:
def fonction_2():
    rout = QRoutine()

    return rout

**Question 7** : Vérifier la réponse à la question 5 en simulant le circuit ci-dessus pour toutes les valeurs possibles de $x$ en entrée

### Oracle de marquage

Maintenant que nous avons implémenté les fonctions $f1$ et $f2$ qui, pour rappel, appliquent la transformation suivante (pour différents cas de recherche)

$$
f(x) = \begin{cases} 0 & \textrm{si x n'est pas recherché} \\ 1 & \textrm{si x est recherché} \end{cases},
$$

nous devons implémenter un oracle $U$ tel que

$$
U(x)= (-1)^{f(x)}x.
$$

Formulé autrement, on veut appliquer une phase de $-1$ quand $f(x)=1$, donc que l'élément $x$ est recherché, sinon rien. On peut alors utilisé une porte quantique de base de l'informatique quantique qui transforme

- $\left| 0 \right \rangle \rightarrow \left| 0 \right \rangle$
- $\left| 1 \right \rangle \rightarrow -\left| 1 \right \rangle$.

La porte Z est adéquate pour effectuer cette transformation. L'idée pour créer l'oracle de marquage en utilisant une porte Z est la suivante

0) l'état initial est $\left| x \right \rangle \left| y=0 \right \rangle$
1) On applique la fonction $f$ pour obtenir $\left| x \right \rangle \left| f(x) \right \rangle$
2) On applique une porte Z sur le qubit $y$, qui a pour effet de rajouter une phase sur les états marqués uniquement
3) On applique l'inverse de la fonction $f$ pour réinitialiser le qubit $y$ dans l'état 0

On a alors en sortie de l'oracle

- Appliquer une phase quand nécessaire pour marquer les états
- Un qubit ancillaire $y$ toujours à 0, que l'on peut ignorer pour la suite

Pour appeler l'inverse d'une routine quantique dans notre implémentation on peut utiliser la méthode *dag* comme suit :

<center>
    ma_routine.dag()(qubits)
</center>


**Question 8** : Implémenter la routine *oracle_marquage* en suivant les étapes détaillées ci-dessus

In [ ]:
def oracle_marquage(fonction):
    rout = QRoutine()

    return rout

**Question 9** : Vérifier à l'aide d'une simulation que l'oracle de marquage applique bien une phase de $-1$ sur les amplitudes des états marqués de la fonction 1 et de la fonction 2

In [ ]:
# Fonction 1

In [ ]:
# Fonction 2

## Amplification

L'objectif est désormais d'amplifier les états marqués, à l'aide de l'opérateur de diffusion. L'opérateur de diffusion sur $n$ qubits correspond au circuit quantique suivant

<center>
    <img src="https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/circuit_diffusion.png" width="450"/>
</center>

**Question 10** : Implémenter la routine quantique operateur de diffusion ayant pour paramètre $n$

In [ ]:
def operateur_de_diffusion(n):
    rout = QRoutine()

    return rout

## Algorithme de Grover

Maintenant que nous possédons toutes les routines quantiques nécessaires, nous pouvons les créer la routine de l'algorithme de Grover. Pour rappel, le circuit quantique correspondant à l'algorithme de Grover est le suivant

<center>
    <img src="https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/grover_circuit.png" width=600/>
</center>

**Question 11**: Créer la routine quantique *grover* qui prend en entrée un *oracle* et un nombre de qubit *n*

In [ ]:
def grover(oracle, n):
    rout = QRoutine()

    return rout

Nous pouvons alors vérifier qu'on trouve bien en sortie de l'algorithme les états recherchés pour la fonction 1 et 2

In [ ]:
n = 4

oracle_1 = oracle_marquage(fonction_1())
circuit_1 = grover(oracle_1, n+1).to_circ()

display_result(circuit_1, idx=[0,1,2,3])

In [ ]:
n = 4

oracle_2 = oracle_marquage(fonction_2())
circuit_2 = grover(oracle_2, n+1).to_circ()

display_result(circuit_2, idx=[0,1,2,3])